In [1]:
import pandas as pd

In [2]:
# Load file
datafile = "Resources/HeroesOfPymoli_Resources_purchase_data.csv"

In [3]:
# Read Purchasing File and store into Pandas data frame
purchasedatadf = pd.read_csv(datafile)

## Display Player Count

In [4]:
numberofplayers = purchasedatadf["SN"]
numplayers = len(numberofplayers.unique().tolist())
pd.DataFrame({"Total Players" : [numplayers]})

,Total Players
0,576


## Purchasing Analysis

In [5]:
numberitems = purchasedatadf["Item ID"]
numuniqueitems = len(numberitems.unique().tolist())

price = purchasedatadf["Price"]
avgprice = sum(price)/len(price)

numberpurchases = purchasedatadf["Item ID"]
numpurchases = numberpurchases.count()

revenue = sum(price)
revenue

purchanalysisdf = pd.DataFrame({"Number of Unique Items" : [numuniqueitems], 
                                "Average Price" : [avgprice],
                                "Number of Purchases" : [numpurchases], 
                                "Total Revenue" : [revenue]})
purchanalysisdf.style.format({'Number of Unique Items': "{}",
                              'Average Price': "${:.2f}",
                              'Number of Purchases': "{}",
                              "Total Revenue":"${:,.2f}"})

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

In [6]:
# Group purchase_data by Gender
genderstats = purchasedatadf.groupby("Gender").nunique()
gendercounts = genderstats["SN"]

percentagegender = (gendercounts / numplayers)*100

genderdemographicsdf = pd.DataFrame({"Total Count": gendercounts, 
                                     "Percentage of Players": percentagegender}).style.format({'Percentage of Players':"{0:.2f}%"})
genderdemographicsdf.index.name = None
genderdemographicsdf



,Total Count,Percentage of Players
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


## Purchasing Analysis (Gender)

In [7]:
avgtotalperson = purchasedatadf.groupby("Gender")["Price"].sum()/1
avgtotalperson


Gender
Female                    361.94
Male                     1967.64
Other / Non-Disclosed      50.19
Name: Price, dtype: float64

In [8]:
purchasecount = purchasedatadf.groupby("Gender")["Purchase ID"].count()
avgpurchaseprice = purchasedatadf.groupby("Gender")["Price"].sum()/purchasecount
totalpurchaseval = purchasedatadf.groupby("Gender")["Price"].sum()
avgpurchaseperson = totalpurchaseval/gendercounts

genderdemographics = pd.DataFrame({"Purchase Count": purchasecount, 
                                    "Average Purchase Price": avgpurchaseprice,
                                    "Total Purchase Value": totalpurchaseval,
                                   "Average Total Purchase per Person": avgpurchaseperson
                                    })


genderdemographics.style.format({"Average Purchase Price":"${:.2f}",
                                 "Total Purchase Value":"${:.2f}","Average Total Purchase per Person":"${:.2f}"})



,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [9]:
binsage = [0,9,14,19,24,29,34,39,100]
labelsage = ["<10","10-14","15-19","20-24","25,29","30-34","35-40","40+"]



In [10]:
purchasedatadf["Age Groups"] = pd.cut(purchasedatadf["Age"], bins=binsage, labels=labelsage)




In [11]:
agegroupeddf = purchasedatadf.groupby("Age Groups")
accuratecountage = agegroupeddf["SN"].nunique()

percentagegroupage = (accuratecountage/numplayers) * 100

agedemographics = pd.DataFrame({"Total Count": accuratecountage,"Percentage of Players": percentagegroupage})
agedemographics.index.name = None
agedemographics.style.format({"Percentage of Players":"{0:.2f}%"})





,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
"25,29",77,13.37%
30-34,52,9.03%
35-40,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

In [12]:
purchasedatadf["Age Groups"] = pd.cut(purchasedatadf["Age"], bins=binsage, labels=labelsage)


In [13]:
agegroupeddf = purchasedatadf.groupby("Age Groups")
totalcountage = agegroupeddf["Price"].count()

totalpurchasevalage = purchasedatadf.groupby("Age Groups")["Price"].sum()
avgpurchasepersonage = totalpurchasevalage/totalcountage
avgpurchaseperson = totalpurchasevalage/totalcountage

In [14]:
purchasinganalysisdf = pd.DataFrame({"Purchase Count": totalcountage, 
                                    "Average Purchase Price": avgpurchasepersonage,
                                    "Total Purchase Value": totalpurchasevalage,
                                     "Average Total Purchase per Person": avgpurchasepersonage
                                    })

purchasinganalysisdf.style.format({"Average Purchase Price":"${:.2f}",
                                   "Total Purchase Value":"${:.2f}","Average Total Purchase per Person":"${:.2f}"})


,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Age Groups,,,,
<10,23,$3.35,$77.13,$3.35
10-14,28,$2.96,$82.78,$2.96
15-19,136,$3.04,$412.89,$3.04
20-24,365,$3.05,$1114.06,$3.05
"25,29",101,$2.90,$293.00,$2.90
30-34,73,$2.93,$214.00,$2.93
35-40,41,$3.60,$147.67,$3.60
40+,13,$2.94,$38.24,$2.94


## Top Spenders

In [15]:
spenderstotalpurchasevaldf = purchasedatadf.loc[:,["SN","Price"]].groupby("SN").sum().sort_values(by = "Price",ascending = False)
spenderspurchasecountdf = purchasedatadf.loc[:,["SN","Price"]].groupby("SN").count().sort_values(by = "Price",ascending = False)
spendersavgpurchasepricedf = (spenderstotalpurchasevaldf / spenderspurchasecountdf).sort_values(by = "Price",ascending = False)


In [16]:
mergedf = pd.merge(spenderspurchasecountdf,spendersavgpurchasepricedf, on="SN")

topspendersdf = pd.merge(mergedf,spenderstotalpurchasevaldf, on="SN")


In [17]:
test = topspendersdf["Price"] >= 13.10
topspendersdf.loc[test].rename(columns = {"Price_x":"Purchase Count",
                                          "Price_y":"Average Purchase Price",
                                          "Price":"Total Purchase Value"}).style.format({"Average Purchase Price":"${:.2f}",
                                                                                         "Total Purchase Value":"${:.2f}"})



,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Iral74,4,$3.40,$13.62
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [18]:
popularpurchasevaldf = purchasedatadf.loc[:,["Item ID","Price"]].groupby("Item ID").sum()
popularpurchasecountdf = purchasedatadf.loc[:,["Item ID","Price"]].groupby("Item ID").count()
popularitemnamesdf = purchasedatadf.loc[:,["Item ID", "Item Name"]]
populartotalpurchasedf = (popularpurchasevaldf / popularpurchasecountdf)


In [19]:
mergepopular1df = pd.merge(popularitemnamesdf,popularpurchasecountdf, on="Item ID")
mergepopular2df = pd.merge(mergepopular1df,popularpurchasevaldf, on="Item ID")
mergepopular3df = pd.merge(mergepopular2df,populartotalpurchasedf, on="Item ID")
popularitemsfinaldf = mergepopular3df.groupby(["Item ID","Item Name"]).mean().sort_values(by = "Price_x",ascending = False)
popularitemsreorderdf = popularitemsfinaldf[["Price_x","Price","Price_y"]]


In [20]:
renamedpopularitemsdf = popularitemsreorderdf.head().rename(columns={"Price_x":"Purchase Count",
                                                                "Price":"Item Price",
                                                              "Price_y":"Total Purchase Value"
                                                             }).style.format({"Item Price":"${:.2f}",
                                                                "Total Purchase Value":"${:.2f}"})
renamedpopularitemsdf


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items


In [21]:
popularitemsfinaldf = mergepopular3df.groupby(["Item ID","Item Name"]).mean().sort_values(by = "Price_y",ascending = False)
popularitemsreorderdf = popularitemsfinaldf[["Price_x","Price","Price_y"]]




In [22]:
renamedprofitableitemsdf = popularitemsreorderdf.head().rename(columns={"Price_x":"Purchase Count",
                                                                "Price":"Item Price",
                                                              "Price_y":"Total Purchase Value"
                                                             }).style.format({"Item Price":"${:.2f}",
                                                                "Total Purchase Value":"${:.2f}"})
renamedprofitableitemsdf


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
